In [3]:
# Simple CNN model for CIFAR-10
import numpy
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
import scipy
from keras.metrics import categorical_accuracy
from sklearn.metrics import accuracy_score
from keras.models import model_from_json
from keras.wrappers.scikit_learn import KerasClassifier

from keras import backend as K
K.set_image_dim_ordering('th')


Using TensorFlow backend.


In [4]:
!pip install h5py

In [5]:
#Default values of the following parameters #batch_size,nb_epochs,nb_classes,split,lr,optimizer,lossfunction,
#activation,dropout
#defaults and constants
batch_size = 128
nb_epochs = 20
nb_classes = 10
split = 0.2
lrate = 0.001
#optimizer = SGD
loss = "categorical_crossentropy"
activation = "relu"
dropout = 0.2
noofneurons=512


In [6]:
# fix random seed for reproducibility
seed = 5
numpy.random.seed(seed)

In [7]:
# load data
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [8]:
# normalize inputs from 0-255 to 0.0-1.0
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

In [9]:
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
#num_classes = y_test.shape[1]

In [10]:
#Model 1

In [11]:
# Create the model
def createmodel1(batchsize,epochs,num_classes,lrate,dropout):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(3, 32, 32), padding='same', activation='relu', kernel_constraint=maxnorm(3)))
    model.add(Dropout(dropout))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    # Compile model
    epochs = epochs
    lrate = lrate
    decay = lrate/epochs
    sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    print(model.summary())
    # Fit the model
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=batchsize)
    # Final evaluation of the model
    scores = model.evaluate(X_test, y_test, verbose=0)
    print("Accuracy: %.2f%%" % (scores[1]*100))

In [12]:
#Model 2

In [13]:
#Deep version of the simple CNN as shownabove.Introducing an additional round of convolutions with many more feature maps.
#Using the same pattern of Convolutional, Dropout, Convolutional and Max Pooling layers.# Create the model
def createmodel2(batchsize,epochs,num_classes,lrate,dropout):
    model2 = Sequential()
    model2.add(Conv2D(32, (3, 3), input_shape=(3, 32, 32), activation='relu', padding='same'))
    model2.add(Dropout(dropout))
    model2.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model2.add(MaxPooling2D(pool_size=(2, 2)))
    model2.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model2.add(Dropout(dropout))
    model2.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model2.add(MaxPooling2D(pool_size=(2, 2)))
    model2.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model2.add(Dropout(dropout))
    model2.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model2.add(MaxPooling2D(pool_size=(2, 2)))
    model2.add(Flatten())
    model2.add(Dropout(dropout))
    model2.add(Dense(1024, activation='relu', kernel_constraint=maxnorm(3)))
    model2.add(Dropout(dropout))
    model2.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
    model2.add(Dropout(dropout))
    model2.add(Dense(num_classes, activation='softmax'))
    # Compile model
    epochs = epochs #25
    lrate = lrate
    decay = lrate/epochs
    sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
    model2.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    print(model2.summary())
    numpy.random.seed(seed)
    model2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=batchsize)
    # Final evaluation of the model
    scores = model2.evaluate(X_test, y_test, verbose=0)
    print("Accuracy: %.2f%%" % (scores[1]*100))
    
    

        

In [24]:
#Model 3
#Deep CNN  with Maxpooling and leakyrelu

from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, advanced_activations
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D

import os
import pickle
import numpy as np



def createmodel3(batchsize,epochs,num_classes,lrate,dropout):
    
# The data, shuffled and split between train and test sets:
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    #print('x_train shape:', x_train.shape)
    #print(x_train.shape[0], 'train samples')
    #print(x_test.shape[0], 'test samples')

    # Convert class vectors to binary class matrices.
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)

    model_3 = Sequential()

    model_3.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
    model_3.add(advanced_activations.LeakyReLU(alpha=0.3))
    model_3.add(Conv2D(32, (3, 3)))
    model_3.add(advanced_activations.LeakyReLU(alpha=0.3))
    model_3.add(MaxPooling2D(pool_size=(2, 2)))
    model_3.add(Dropout(0.25))

    model_3.add(Conv2D(64, (3, 3), padding='same'))
    model_3.add(advanced_activations.LeakyReLU(alpha=0.3))
    model_3.add(Conv2D(64, (3, 3)))
    model_3.add(advanced_activations.LeakyReLU(alpha=0.3))
    model_3.add(MaxPooling2D(pool_size=(2, 2)))
    model_3.add(Dropout(0.25))

    model_3.add(Flatten())
    model_3.add(Dense(512))
    model_3.add(advanced_activations.LeakyReLU(alpha=0.3))
    model_3.add(Dropout(0.5))
    model_3.add(Dense(num_classes))
    model_3.add(Activation('softmax'))

    # initiate RMSprop optimizer
    opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

    # Let's train the model using RMSprop
    model_3.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255

    if not data_augmentation:
        print('Not using data augmentation.')
        model_1A.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
    else:
        print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
        datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(X_train)

    # Fit the model on the batches generated by datagen.flow().
    model_3.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

    # Load label names to use in prediction results
    label_list_path = 'datasets/cifar-10-batches-py/batches.meta'


    keras_dir = os.path.expanduser(os.path.join('~', '.keras'))
    datadir_base = os.path.expanduser(keras_dir)
    if not os.access(datadir_base, os.W_OK):
        datadir_base = os.path.join('/tmp', '.keras')
    label_list_path = os.path.join(datadir_base, label_list_path)

    with open(label_list_path, mode='rb') as f:
        labels = pickle.load(f)

# Evaluate model with test data set and share sample prediction results
    evaluation = model_3.evaluate_generator(datagen.flow(x_test, y_test,
                                                   batch_size=batch_size,
                                                   shuffle=False),
                                      steps=x_test.shape[0] // batch_size,
                                      workers=4)
    print('Model Accuracy = %.2f' % (evaluation[1]))

    predict_gen = model_3.predict_generator(datagen.flow(x_test, y_test,
                                                   batch_size=batch_size,
                                                   shuffle=False),
                                      steps=x_test.shape[0] // batch_size,
                                      workers=4)

    for predict_index, predicted_y in enumerate(predict_gen):
        actual_label = labels['label_names'][np.argmax(y_test[predict_index])]
        predicted_label = labels['label_names'][np.argmax(predicted_y)]
        print('Actual Label = %s vs. Predicted Label = %s' % (actual_label,
                                                          predicted_label))
        if predict_index == num_predictions:
            break

In [25]:

from io import StringIO
import requests
import json
import pandas as pd

# @hidden_cell
# This function accesses a file in your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def get_object_storage_file_with_credentials_f2b31d3aa8f74da9a868a4102976b52e(container, filename):
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage."""

    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': 'member_eaa7a3ccbee220b104765fd6ecbda6648009a26f','domain': {'id': 'a76f4599636a4d9fb2a2209d58f8cb53'},
            'password': 'f=O00,1/BLCkVq-Q'}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', container, '/', filename])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.get(url=url2, headers=headers2)
    return StringIO(resp2.text)

df = pd.read_csv(get_object_storage_file_with_credentials_f2b31d3aa8f74da9a868a4102976b52e('DefaultProjectlekhrajanijhuskyneuedu', 'Inputs.csv'))
df.head()


,batchsize,epochs,num_classes,lrate,dropout
0,32,25,10,0.010,0.20
1,64,25,10,0.010,0.20
2,32,20,10,0.012,0.25
3,32,25,10,0.010,0.20


In [16]:
# Training Model 1 on the dataset(Activation= Relu&Softmax, Optimizer=SGD())Basic CNN
def createmodels(df):
    for index, row in df.iterrows():
        createmodel1(row['batchsize'].astype(int),row['epochs'].astype(int),row['num_classes'].astype(int),row['lrate'],row['dropout'])

In [ ]:
createmodels(df)


In [17]:
#Training Model 2 on dataset(Activation= Relu&Softmax, Optimizer=SGD())Deep CNN
def createmodels(df):
    for index, row in df.iterrows():
        createmodel2(row['batchsize'].astype(int),row['epochs'].astype(int),row['num_classes'].astype(int),row['lrate'],row['dropout'])

In [ ]:
createmodels(df)


In [ ]:
#Training Model 3 on dataset(Activation=Relu&Softmax, Optimizer=RMSprop())

In [23]:
def createmodels(df):
    for index, row in df.iterrows():
        createmodel3(row['batchsize'],row['epochs'],row['num_classes'],row['lrate'],row['dropout'])

In [ ]:
createmodels(df)


In [19]:
#Assignment 1 Part 2C

In [17]:
X=np.concatenate((X_train, X_test), axis=0)
y=np.concatenate((y_train,y_test), axis=0)

In [18]:
from sklearn.model_selection import train_test_split
random_state=5
X_train2c, X_test2c, y_train2c, y_test2c = train_test_split(X, y, test_size=0.2, random_state=5)

In [19]:
label_list_path = 'datasets/cifar-10-batches-py/batches.meta'


In [20]:
#generating labels(code taken from the base code)
keras_dir = os.path.expanduser(os.path.join('~', '.keras'))
datadir_base = os.path.expanduser(keras_dir)
if not os.access(datadir_base, os.W_OK):
    datadir_base = os.path.join('/tmp', '.keras')
label_list_path = os.path.join(datadir_base, label_list_path)

with open(label_list_path, mode='rb') as f:
    labels = pickle.load(f)

In [21]:
from scipy.stats import itemfreq
a= itemfreq(y_test2c)
for i in range (0,len(a)):
    print(labels['label_names'][i],a[i][1])

airplane 108000.0
automobile 12000.0


In [22]:
from collections import Counter
Counter(y_test2c[:,0])

Counter({0.0: 10793, 1.0: 1207})

In [28]:
from keras.metrics import categorical_accuracy
from sklearn.metrics import accuracy_score
from keras.models import model_from_json

In [38]:
model2 = Sequential()
model2.add(Conv2D(32, (3, 3), input_shape=(3, 32, 32), activation='relu', padding='same'))
model2.add(Dropout(0.2))
model2.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model2.add(Dropout(0.2))
model2.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model2.add(Dropout(0.2))
model2.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Flatten())
model2.add(Dropout(0.2))
model2.add(Dense(1024, activation='relu', kernel_constraint=maxnorm(3)))
model2.add(Dropout(0.2))
model2.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
model2.add(Dropout(0.2))
model2.add(Dense(10, activation='softmax'))
    # Compile model
epochs = 25
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model2.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model2.summary())
numpy.random.seed(seed)
model2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=64)
model2.save('model2.h5')
    # Final evaluation of the model


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
dropout_13 (Dropout)         (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 32, 16, 16)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 64, 16, 16)        18496     
_________________________________________________________________
dropout_14 (Dropout)         (None, 64, 16, 16)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 64, 16, 16)        36928     
__________

In [39]:
scores = model2.evaluate(X_test2c, y_test2c, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 90.75%


In [ ]:
#Second method

In [ ]:
from Models import model1,model2,model3


In [40]:
#Deep version of the simple CNN as shownabove.Introducing an additional round of convolutions with many more feature maps.
#Using the same pattern of Convolutional, Dropout, Convolutional and Max Pooling layers.# Create the model
def createAndSaveModel(batchsize,epochs,num_classes,lrate,dropout,modelname,weights):
    model2 = Sequential()
    model2.add(Conv2D(32, (3, 3), input_shape=(3, 32, 32), activation='relu', padding='same'))
    model2.add(Dropout(dropout))
    model2.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model2.add(MaxPooling2D(pool_size=(2, 2)))
    model2.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model2.add(Dropout(dropout))
    model2.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model2.add(MaxPooling2D(pool_size=(2, 2)))
    model2.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model2.add(Dropout(dropout))
    model2.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model2.add(MaxPooling2D(pool_size=(2, 2)))
    model2.add(Flatten())
    model2.add(Dropout(dropout))
    model2.add(Dense(1024, activation='relu', kernel_constraint=maxnorm(3)))
    model2.add(Dropout(dropout))
    model2.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
    model2.add(Dropout(dropout))
    model2.add(Dense(num_classes, activation='softmax'))
    # Compile model
    epochs = epochs #25
    lrate = lrate
    decay = lrate/epochs
    sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
    model2.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    print(model2.summary())
    numpy.random.seed(seed)
    model2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=batchsize)
    # Final evaluation of the model
    scores = model2.evaluate(X_test, y_test, verbose=0)
    print("Accuracy: %.2f%%" % (scores[1]*100))
    
    #save model to json format
    modeljson = model2.to_json()
    open(modelname, 'w').write(modeljson)

    #Save the weights learnt
    model2.save_weights(weights,overwrite=True)

        

In [ ]:
createAndSaveModel(*model2)

In [ ]:
json_file = open("model2_cifar10.json.json", "r")
model2loaded = json_file.read()
son_file.close()
modeltobetested = model_from_json(model2loaded)
modeltobetested.load_weights("model2_cifar10_weights.h5")
# Evaluate model on test data
predictedvalues = loaded_model.predict_classes(X_test2c)
print(predictedvalues)



In [ ]:
#Model Accuracy when trained using 12,000 images instead of running the best model that has been built earlier

In [44]:
#Generating 12000 random traing and testing data from orignal data
x_total =np.concatenate((X_train,X_test),axis=0)
y_total =np.concatenate((y_train,y_test),axis=0)
seed = 5
np.random.seed(seed)
random = np.random.randint(0,60000,size=12000)
x_train_new = x_total[random]
y_train_new = y_total[random]

In [45]:
#summary of the images 
a= itemfreq(y_train_new)
for i in range (0,len(a)):
    print(labels['label_names'][i],a[i][1])

airplane 108000.0
automobile 12000.0


In [47]:
modelnew = Sequential()
modelnew.add(Conv2D(32, (3, 3), input_shape=(3, 32, 32), activation='relu', padding='same'))
modelnew.add(Dropout(0.2))
modelnew.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
modelnew.add(MaxPooling2D(pool_size=(2, 2)))
modelnew.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
modelnew.add(Dropout(0.2))
modelnew.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
modelnew.add(MaxPooling2D(pool_size=(2, 2)))
modelnew.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
modelnew.add(Dropout(0.2))
modelnew.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
modelnew.add(MaxPooling2D(pool_size=(2, 2)))
modelnew.add(Flatten())
modelnew.add(Dropout(0.2))
modelnew.add(Dense(1024, activation='relu', kernel_constraint=maxnorm(3)))
modelnew.add(Dropout(0.2))
modelnew.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
modelnew.add(Dropout(0.2))
modelnew.add(Dense(10, activation='softmax'))
# Compile model
epochs = 25
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
modelnew.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(modelnew.summary())
numpy.random.seed(seed)
modelnew.fit(x_train_new, y_train_new,validation_split=0.2, epochs=epochs, batch_size=64)
modelnew.save('modelnew.h5')


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_31 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
dropout_31 (Dropout)         (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 32, 16, 16)        0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 64, 16, 16)        18496     
_________________________________________________________________
dropout_32 (Dropout)         (None, 64, 16, 16)        0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 64, 16, 16)        36928     
__________